In [ ]:
import re
import os
import argparse
import shutil
import numpy as np

import torch
from torch.autograd import Variable

from data_utils import *

In [ ]:
class seq2seqAttention(torch.nn.Module):
    
    def __init__(
        self,
        src_emb_dim=100,
        trg_emb_dim=100,
        src_hidden_dim=25,
        trg_hidden_dim=50,
        src_vocab_size=999,
        trg_vocab_size=999,
        src_pad_token=0,
        trg_pad_token=0,
        src_nlayer=2,
        trg_nlayer=1,
        src_bidirect=True,
        batch_size=128,
        dropout=0.0
    ):
        super(seq2seqAttention, self).__init__()
        # source embedding
        self.src_embedding = torch.nn.Embedding(
            src_vocab_size,
            src_emb_dim,
            padding_idx=0
        ).cuda()
        torch.nn.init.normal(
            self.src_embedding.weight, 
            mean=0.0, 
            std=0.02
        )
        # targe embedding
        self.trg_embedding = torch.nn.Embedding(
            trg_vocab_size,
            trg_emb_dim,
            padding_idx=0
        ).cuda()
        torch.nn.init.normal(
            self.trg_embedding.weight,
            mean=0.0,
            std=0.02
        )
        
    def forward(self):
        
        return
    
model = seq2seqAttention(
    src_emb_dim=100,
    trg_emb_dim=100,
    src_hidden_dim=25,
    trg_hidden_dim=50,
    src_vocab_size=len(vocab2id),
    trg_vocab_size=len(vocab2id),
    src_pad_token=0,
    trg_pad_token=0,
    src_nlayer=2,
    trg_nlayer=1,
    src_bidirect=True,
    batch_size=batch_size,
    dropout=0.0
).cuda()

print model

In [12]:
data_dir = '../sum_data/'
file_vocab = 'cnn_vocab.txt'
file_corpus = 'cnn.txt'
n_epoch = 20
batch_size = 64

vocab2id, id2vocab = construct_vocab(data_dir+'/'+file_vocab)
print 'The vocabulary size: {0}'.format(len(vocab2id))

n_batch = create_batch_file(file_name='../sum_data/cnn.txt', batch_size=batch_size)
print 'The number of batches: {0}'.format(n_batch)



The vocabulary size: 80475
The number of batches: 1444


In [71]:
import argparse
import shutil
import re
import os

import torch
from torch.autograd import Variable

from lstm2lstm import *
from data_utils import *

data_dir = '../sum_data/'
file_vocab = 'cnn_vocab.txt'
file_corpus = 'cnn.txt'
n_epoch = 20
batch_size = 64

vocab2id, id2vocab = construct_vocab(data_dir+'/'+file_vocab)
print 'The vocabulary size: {0}'.format(len(vocab2id))

n_batch = create_batch_file(file_name='../sum_data/cnn.txt', batch_size=batch_size)
print 'The number of batches: {0}'.format(n_batch)

model = seq2seq(
    src_emb_dim=100,
    trg_emb_dim=100,
    src_hidden_dim=25,
    trg_hidden_dim=50,
    src_vocab_size=len(vocab2id),
    trg_vocab_size=len(vocab2id),
    src_pad_token=0,
    trg_pad_token=0,
    src_nlayer=2,
    trg_nlayer=1,
    src_bidirect=True,
    batch_size=batch_size,
    dropout=0.0
).cuda()

model.load_state_dict(torch.load('../sum_data/lstm2lstm_results/lstm2lstm_15_1400.model'))

src_var, trg_input_var, trg_output_var = process_minibatch(
    100, vocab2id, max_lens=[512, 64]
)

trg_input_arr = [[vocab2id['<s>'] for k in range(64)] for j in range(batch_size)]
trg_input_var = Variable(torch.LongTensor(trg_input_arr))
for k in range(64):
    print k,
    logits = model(src_var.cuda(), trg_input_var.cuda())
    word_prob = model.decode(logits).data.cpu().numpy().argmax(axis=-1)
    for j in range(64):
        trg_input_arr[j][k] = word_prob[j][k]
        trg_input_var = Variable(torch.LongTensor(trg_input_arr))
print

The vocabulary size: 80475
The number of batches: 1444
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63


In [73]:
idid = 32
sen_pred = [id2vocab[x] for x in word_prob[idid]]
print ''.join(['-' for k in range(50)])
st_idx = len(sen_pred)
for k, wd in enumerate(sen_pred):
    if wd == '</s>':
        st_idx = k
        break
sen_pred = sen_pred[:st_idx]
print ' '.join(sen_pred)

print ''.join(['-' for k in range(50)])
sen_abs = [id2vocab[x] for x in trg_output_var.data[idid]]
st_idx = len(sen_abs)
for k, wd in enumerate(sen_abs):
    if wd == '<pad>':
        st_idx = k
        break
print ' '.join(sen_abs[:st_idx])

print ''.join(['-' for k in range(50)])
sen_source = [id2vocab[x] for x in src_var.data[idid]]
st_idx = len(sen_source)
for k, wd in enumerate(sen_source):
    if wd == '<pad>':
        st_idx = k
        break
print ' '.join(sen_source[:st_idx])

--------------------------------------------------
york the the he `` `` `` `` `` `` `` the `` the `` the the the but obama 's the he obama `` `` `` but `` the will not world the he `` the but his the the not world < `` the but his the the not world the the <unk>
--------------------------------------------------
new : tutu urges <unk> to cool burial dispute . mandela 's family had weighed taking him off life support , document states . court papers filed in a dispute over burials call mandela 's health `` perilous '' . three of mandela 's deceased children are reburied at his home in <unk> . </s>
--------------------------------------------------
<s> former south african president nelson mandela 's health had declined so sharply last week that his family was considering whether to take him off life support before his condition improved , a court document released thursday revealed . the document , known as a `` certificate of urgency , '' was filed on june 26 . it stated that the 94-y